There are multiple questions that are going in my head.

1. does win rate change over time? why?

2. Nasir club was the champion of the 2018/2019 season, Al hilal is the 2019/2020 season. What makes a champ?

3. does formation matter?

4. how important is the coach?

5. does the main scorrer participation in small clubs make as much difference as big clubs?

6. does playing in small unpopular venues make any difference?

7. how does previous match rating affect the next match result?

8. What is more relative, last matche rating, or the last match against the opposing team rating?

9. Does playing in unpopular venues have an effect on peformance?



In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split

np.random.seed(42)
%matplotlib inline

In [64]:
season2018 = pd.read_csv("Season2018.csv").drop("Unnamed: 0", axis=1)
season2019 = pd.read_csv("Season2019.csv").drop("Unnamed: 0", axis=1)
season2020 = pd.read_csv("Season2020.csv")

df = pd.concat([season2018, season2019, season2020]).reset_index(drop = True)

In [65]:
df.head()

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
0,151695,Al-Ettifaq,Al-Raed,1.0,1.0,Prince Mohamed Bin Fahd Stadium,NaN,2018-08-30T18:40:00+03:00,2018/2019,L. Ramos,B. Hasi,4-2-3-1,4-2-3-1,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...","[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",1,Match Finished,7.00,6.98
1,151696,Al-Hazm,Al Wehda Club,0.0,0.0,King Abdul Aziz Stadium,NaN,2018-08-30T19:05:00+03:00,2018/2019,D. Isăilă,Fábio Carille,4-2-3-1,4-4-2,"[('M. Asselah', None), ('Alemão', None), ('Kha...","[('Mohamed Awad', None), ('Osama Hawsawi', Non...",1,Match Finished,6.97,6.77
2,151697,Ohod,Al-Nassr,1.0,2.0,Prince Mohammed bin Abdul Aziz Stadium,NaN,2018-08-30T19:15:00+03:00,2018/2019,F. Arce,D. Carreño,4-2-3-1,4-2-3-1,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...","[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",1,Match Finished,6.64,6.83
3,151698,Al-Qadisiyah FC,Al-Fateh,0.0,0.0,Prince Saud bin Jalawi Stadium,"Mark Clattenburg, England",2018-08-31T18:40:00+03:00,2018/2019,A. Stanojević,F. Al Jabal,4-4-2,4-2-3-1,"[('J. Duncan', 'G'), ('Adnan Falatah', None), ...","[('M. Koval', 'G'), ('M. Aguirregaray', 'D'), ...",1,Match Finished,6.96,7.01
4,151699,Al-Hilal Saudi FC,Al-Fayha,1.0,0.0,King Fahd Stadium,NaN,2018-08-31T18:55:00+03:00,2018/2019,Jorge Jesus,G. Costas,4-1-4-1,4-3-3,"[('Ali Al Habsi', 'G'), ('Botía', 'D'), ('Yasi...","[('C. Bonilla', 'G'), ('Naldo', 'M'), ('Gêgê',...",1,Match Finished,7.07,6.55


let's first isolate a portion of the matches (test set) so that it doesn't affect our decision making in the exploration phase

* I'll isolate 20% of the data.
* and i'll isolate it so that we have 20% of each season in our test set
* I'll not use indices i'll use fixture_ids


In [66]:
print(df.shape)
print(season2018.shape)
print(season2019.shape)
print(season2020.shape)

(720, 19)
(240, 19)
(240, 19)
(240, 19)


In [86]:
np.random.seed(42)
season2018_test = np.random.permutation(season2018["fixture_id"])[:48]
season2019_test = np.random.permutation(season2019["fixture_id"])[:48]
season2020_test = np.random.permutation(season2020["fixture_id"])[:48]
test_fixtures = np.append(np.append(season2018_test, season2019_test), season2020_test)

In [41]:
len(test_fixtures)

144

In [87]:
test_fixtures[:10]

array([151719, 151701, 151790, 151805, 151800, 151867, 151929, 151783,
       151704, 151838])

In [89]:
test_set = df.loc[df["fixture_id"].isin(test_fixtures)]

In [90]:
len(test_set)

144

In [91]:
df = df.loc[~(df["fixture_id"].isin(test_fixtures))]

In [92]:
len(df)

576

# Initial data exploration

finding tredns & insights, and identifying flaws.

In [101]:
df.head()

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
0,151695,Al-Ettifaq,Al-Raed,1.0,1.0,Prince Mohamed Bin Fahd Stadium,NaN,2018-08-30 18:40:00,2018/2019,L. Ramos,B. Hasi,4-2-3-1,4-2-3-1,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...","[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",1,Match Finished,7.00,6.98
1,151696,Al-Hazm,Al Wehda Club,0.0,0.0,King Abdul Aziz Stadium,NaN,2018-08-30 19:05:00,2018/2019,D. Isăilă,Fábio Carille,4-2-3-1,4-4-2,"[('M. Asselah', None), ('Alemão', None), ('Kha...","[('Mohamed Awad', None), ('Osama Hawsawi', Non...",1,Match Finished,6.97,6.77
2,151697,Ohod,Al-Nassr,1.0,2.0,Prince Mohammed bin Abdul Aziz Stadium,NaN,2018-08-30 19:15:00,2018/2019,F. Arce,D. Carreño,4-2-3-1,4-2-3-1,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...","[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",1,Match Finished,6.64,6.83
3,151698,Al-Qadisiyah FC,Al-Fateh,0.0,0.0,Prince Saud bin Jalawi Stadium,"Mark Clattenburg, England",2018-08-31 18:40:00,2018/2019,A. Stanojević,F. Al Jabal,4-4-2,4-2-3-1,"[('J. Duncan', 'G'), ('Adnan Falatah', None), ...","[('M. Koval', 'G'), ('M. Aguirregaray', 'D'), ...",1,Match Finished,6.96,7.01
4,151699,Al-Hilal Saudi FC,Al-Fayha,1.0,0.0,King Fahd Stadium,NaN,2018-08-31 18:55:00,2018/2019,Jorge Jesus,G. Costas,4-1-4-1,4-3-3,"[('Ali Al Habsi', 'G'), ('Botía', 'D'), ('Yasi...","[('C. Bonilla', 'G'), ('Naldo', 'M'), ('Gêgê',...",1,Match Finished,7.07,6.55


In [93]:
#Let's first do the most basic operations 
df.isnull().sum()

fixture_id         0
home_team          0
away_team          0
home_goals        18
away_goals        18
venue              0
referee           63
date               0
season             0
home_coach        45
away_coach        43
home_formation    31
away_formation    31
home_players      18
away_players      18
round              0
status             0
H_avg_ratings     97
A_avg_ratings     97
dtype: int64

since only 18 matches are 'Uncollected yet' let's subtract 18 from the results. Negative entries mean no missing data

In [94]:
df.isnull().sum() - 18

fixture_id       -18
home_team        -18
away_team        -18
home_goals         0
away_goals         0
venue            -18
referee           45
date             -18
season           -18
home_coach        27
away_coach        25
home_formation    13
away_formation    13
home_players       0
away_players       0
round            -18
status           -18
H_avg_ratings     79
A_avg_ratings     79
dtype: int64

* We have 45 missing referees, 52 coaches if you add home & away teams. 13 formations, 79 ratings (great portion of them is uncollected yet)

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 719
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fixture_id      576 non-null    int64  
 1   home_team       576 non-null    object 
 2   away_team       576 non-null    object 
 3   home_goals      558 non-null    float64
 4   away_goals      558 non-null    float64
 5   venue           576 non-null    object 
 6   referee         513 non-null    object 
 7   date            576 non-null    object 
 8   season          576 non-null    object 
 9   home_coach      531 non-null    object 
 10  away_coach      533 non-null    object 
 11  home_formation  545 non-null    object 
 12  away_formation  545 non-null    object 
 13  home_players    558 non-null    object 
 14  away_players    558 non-null    object 
 15  round           576 non-null    int64  
 16  status          576 non-null    object 
 17  H_avg_ratings   479 non-null    flo

* date column must be converted to datetime

In [97]:
df.describe()

,fixture_id,home_goals,away_goals,round,H_avg_ratings,A_avg_ratings
count,576.000000,558.000000,558.000000,576.000000,479.000000,479.000000
mean,357680.144097,1.553763,1.370968,15.432292,6.865261,6.819833
std,211801.638593,1.256857,1.183527,8.671328,0.230950,0.234582
min,151695.000000,0.000000,0.000000,1.000000,6.100000,6.150000
25%,151874.750000,1.000000,1.000000,8.000000,6.720000,6.675000
50%,312858.500000,1.000000,1.000000,15.000000,6.870000,6.830000
75%,635921.250000,2.000000,2.000000,23.000000,7.020000,6.980000
max,657882.000000,7.000000,6.000000,30.000000,7.500000,7.570000


we notice that the mean of the home team ratings is slightly greater than the away team which might mean that being home or not is a good predictor

In [100]:
df.date

0      2018-08-30 18:40:00
1      2018-08-30 19:05:00
2      2018-08-30 19:15:00
3      2018-08-31 18:40:00
4      2018-08-31 18:55:00
              ...         
714    2021-05-30 21:00:00
715    2021-05-30 21:00:00
716    2021-05-30 21:00:00
718    2021-05-30 21:00:00
719    2021-05-30 21:00:00
Name: date, Length: 576, dtype: object

to possibly change the structure of the data run the following cells

### Little discussion

In [110]:
df.columns

Index(['fixture_id', 'home_team', 'away_team', 'home_goals', 'away_goals',
       'venue', 'referee', 'date', 'season', 'home_coach', 'away_coach',
       'home_formation', 'away_formation', 'home_players', 'away_players',
       'round', 'status', 'H_avg_ratings', 'A_avg_ratings'],
      dtype='object')

I am not sure if this is the optimal format for the data, I am worried that since entries are split between Home & Away, when we want to perform some eda or predict the outcome we would have 2 target variables to work against. 
Another thing is that the machine would probably consider being home or away as significantly different when it is not.
Maybe we should change the format so that we have the following 
* [Team, Current points, IsHome, opponent, opponent current points, Vinue, coach, opponent coach, formation, opponnet formation, average ratings, opponent ratings, Date, Season, referee, Goals Scored]

Now the target variable is goals scored.
we can try to predict the final outcome of the match by running the model twice on both teams, then by comparing the goals we give the final result as Win, defeat, draw. 

# Data Cleaning 

* Fixing the date column

In [104]:
df.loc[0, "date"]

Timestamp('2018-08-30 18:40:00')

In [99]:
df["date"] = [date[:-6].replace("T", " ") for date in df["date"]]

/Users/yazeed/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [103]:
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d %H:%M:%S")


/Users/yazeed/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


* changing the strucutre of the data

In [ ]:
# since every entry now is based on home I'll create a feature called IsHome and set it to 1

df["IsHome"] = 1

In [ ]:
len([x for x in range(0, len(df)*2, 2)])

In [ ]:
len(df)

In [ ]:
df.index = [x for x in range(0, len(df) * 2, 2)]

In [ ]:
df.columns

I'll change the order of the columns so that it is close to the desired format

In [ ]:
df = df[["home", "home_format", "IsHome", "away", "away_format", "players","referee", "season", "home_goals", "away_goals"]]

In [ ]:
for i in [x for x in range(1, len(df) * 2 + 1, 2)]:
    old_entry = df.loc[i-1]
    df.loc[i] = [old_entry.away, old_entry.away_format, 0, old_entry.home, old_entry.home_format, old_entry.players, old_entry.referee, old_entry.season, old_entry.away_goals, np.nan]
    

In [ ]:
df.sort_index(inplace=True)

In [ ]:
# Now we will change the columns so that it aligns with our format

df.drop("away_goals", inplace=True, axis=1)

df.columns = ["Team1", "format", "IsHome", "Opponent", "Opponent_format", "Players", "Referee", "Season", "GoalsScored"]

In [ ]:
df.GoalsScored = df.GoalsScored.astype("int64")

# Feature Engineering

In [ ]:
df["pts"] = 0
df["opponent_pts"] = 0
df["result"] = 0

In [105]:
#Adding the current points feature

for season in ["2018/2019", "2019/2020", "2020/2021"]:
    
    season_df = df.loc[(df["Season"] == season )]

    points = {}

    for team in season_df.home_team.unique():
    points[team] = 0
    
    
    for i in season_df.index:
        if i % 2 == 0 :
            Team1 = season_df.loc[i, "Team1"]
            Team2 = season_df.loc[i, "Opponent"]


            season_df.loc[i,"pts"] = points[Team1]
            season_df.loc[i, "opponent_pts"] = points[Team1]

            season_df.loc[i+1,"pts"] = points[Team2]
            season_df.loc[i+1, "opponent_pts"] = points[Team1]


            if season_df.loc[i, "GoalsScored"] > season_df.loc[i+1, "GoalsScored"]:
                points[Team1] += 3
                season_df.loc[i, "result"] = "Win"
                season_df.loc[i+1, "result"] = "Defeat"
            
            elif season_df.loc[i, "GoalsScored"] == season_df.loc[i+1, "GoalsScored"]:
                points[Team1] += 1
                points[Team2] += 1
                season_df.loc[i, "result"] = "Draw"
                season_df.loc[i+1, "result"] = "Draw"

            else:
                points[Team2] += 3
                season_df.loc[i, "result"] = "Defeat"
                season_df.loc[i+1, "result"] = "Win"

        df.loc[df["Season"] == season] = season_df




In [106]:
# Adding the final state feature

In [107]:
# Adding the last game performance

In [108]:
# Adding the last game agains the other team performance